<a href="https://colab.research.google.com/github/Keishin315/sony_compe/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
stadiumdata=pd.read_csv('stadium_capacity_mapping.csv')
holidaydata=pd.read_csv('holidays_in_japan.csv')
samsubdata=pd.read_csv('sample_submit-4.csv')
match_reportsdata=pd.read_csv('match_reports.csv')
testdata=pd.read_csv('test-5.csv')
traindata=pd.read_csv('train-4.csv')

#同じスタジアムは統一
stadiumdict={'日産スタジアム':'日産スタジアム'}
for i in range(1,len(stadiumdata)):
  if(stadiumdata.iloc[i][1]==stadiumdata.iloc[i-1][1]):
    stadiumdict[stadiumdata.iloc[i][0]]=stadiumdata.iloc[i-1][0]
  else:
    stadiumdict[stadiumdata.iloc[i][0]]=stadiumdata.iloc[i][0]

holidaydata['holiday_date'] = pd.to_datetime(holidaydata['holiday_date'])
#祝日の前後が休みかどうか
from datetime import date,timedelta
holidaydata['zenjitsu']=0
for i in range(len(holidaydata)):
  if(holidaydata.iloc[i][0].weekday()==0 or (holidaydata['holiday_date'][i]-timedelta(days=1) in holidaydata['holiday_date'].values)):
      holidaydata['zenjitsu'][i]=1

holidaydata['yokujitsu']=0
for i in range(len(holidaydata)):
  if(holidaydata.iloc[i][0].weekday()==4 or (holidaydata['holiday_date'][i]+timedelta(days=1) in holidaydata['holiday_date'].values)):
      holidaydata['yokujitsu'][i]=1

for i in range(1,18):
  holidaydata.loc[233+i]=[pd.to_datetime(str(2000+i)+'-08-13'),'obon',0,0]
  holidaydata.loc[234+i]=[pd.to_datetime(str(2000+i)+'-08-14'),'obon',0,0]
  holidaydata.loc[235+i]=[pd.to_datetime(str(2000+i)+'-08-15'),'obon',0,0]
  holidaydata.loc[236+i]=[pd.to_datetime(str(2000+i)+'-08-16'),'obon',0,0]
#スタジアムの編集
for i in range(len(traindata)):
  traindata['venue'][i]=stadiumdict[traindata['venue'][i]]

#capacity->現時点で使用していない
stdic={}
for i in range(len(stadiumdata)):
  stdic[stadiumdict[stadiumdata['stadium'][i]]]=stadiumdata['capacity'][i]



#targetの収容率を計算
traindata['attendance_rate']=0.0
for i in range(len(traindata)):
  traindata['attendance_rate'][i]=traindata['attendance'][i]/stdic[traindata['venue'][i]]

#traindataの編集
for i in traindata.columns:
  print(i,':',traindata[i].nunique())
original=traindata
target = traindata['attendance_rate']
traindata=traindata.drop(['attendance','attendance_rate'],axis=1)
#datetime処理
testdata['match_date']=pd.to_datetime(testdata['match_date'])
testdata['kick_off_time']=pd.to_datetime(testdata['kick_off_time'])
traindata['match_date']=pd.to_datetime(traindata['match_date'])
testdata['match_date']=pd.to_datetime(testdata['match_date'])
traindata['kick_off_time']=pd.to_datetime(traindata['kick_off_time'])
testdata['kick_off_time']=pd.to_datetime(testdata['kick_off_time'])
#祝日かどうか
traindata['holiday_flag']=0
for i in range(len(traindata)):
  if(traindata['match_date'][i] in holidaydata['holiday_date'].values):
    traindata['holiday_flag'][i]=1
  else:
    traindata['holiday_flag'][i]=0
#前後の日が休みかどうか
traindata['zenjitsu']=0
for i in range(len(traindata)):
  if(traindata.iloc[i]['match_date'].weekday()==0 or traindata.iloc[i]['match_date'].weekday()==1 or (traindata['match_date'][i]-timedelta(days=1) in holidaydata['holiday_date'].values)):
      traindata['zenjitsu'][i]=1

traindata['yokujitsu']=0
for i in range(len(traindata)):
  if(traindata.iloc[i]['match_date'].weekday()==4 or traindata.iloc[i]['match_date'].weekday()==5 or (traindata['match_date'][i]+timedelta(days=1) in holidaydata['holiday_date'].values)):
      traindata['yokujitsu'][i]=1
traindata['homescore']=match_reportsdata['home_team_score'][:3366]
traindata['awayscore']=match_reportsdata['away_team_score'][:3366]

#日付の追加
import re
traindata['month']=0
traindata['year']=0
traindata['day']=0
traindata['hour']=0
traindata['weekday']=0
for i in range(len(traindata)):
  traindata['month'][i]=traindata.loc[i]['match_date'].month
  traindata['year'][i]=traindata.loc[i]['match_date'].year-2000
  traindata['day'][i]=traindata.loc[i]['match_date'].day
  traindata['hour'][i]=traindata.loc[i]['kick_off_time'].hour
  traindata['weekday'][i]=traindata.loc[i]['match_date'].weekday()
  traindata['section'][i]=int(re.split('第|節',traindata['section'][i])[1])
  traindata['round'][i]=int(re.split('第|日',traindata['round'][i])[1])

#2015,2016のsectionの誤りを修正
flag=0
for i in range(len(traindata)-1):
  if(traindata['section'][i]==17 and traindata['section'][i+1]==1 and flag==0):
    flag=1
    continue
  elif(traindata['section'][i]==17 and traindata['section'][i+1]==1 and flag==1):
    flag=0
    traindata['section'][i]+=17
  if(flag):
    traindata['section'][i]+=17
  
  
traindata['section'][3365]+=17

#勝敗
traindata['winningteam']=0
for i in range(len(traindata)):
  if(traindata['homescore'][i]>traindata['awayscore'][i]):
    traindata['winningteam'][i]=0
  elif(traindata['homescore'][i]<traindata['awayscore'][i]):
    traindata['winningteam'][i]=2
  else:
    traindata['winningteam'][i]=1

#天気
for condition in ["晴", "曇", "雨", "屋内", "のち", "時々", "一時"]:
    traindata["weather_{}".format(condition)] = traindata["weather"].str.contains(condition)
    traindata["weather_その他"] = traindata["weather"].str.contains("雪|雷|霧")
#対戦カード

teams = traindata["home_team"].unique().tolist()
teams.append('長崎')

for team in teams:
  traindata["team_" + team] = ((traindata["home_team"] == team)|
                                        (traindata["away_team"] == team))




#順位表のシミュレーション
table = pd.DataFrame([teams,[],[]]).T
table[1],table[2]=0,0
table=table.set_index(0)
table
tables=[]
def rank_upgrade(home,away,res):
  if(res==2):
    table[1][away]+=3
  elif(res==1):
    table[1][home]+=1
    table[1][away]+=1
  elif(res==0):
    table[1][home]+=3
  
  kat=sorted(table[1],reverse=True)
  for t in teams:
    table[2][t]=kat.index(table[1][t])+1
  
def rank_reset():
  table[1]=0

traindata['homerank'],traindata['awayrank']=0,0
for i in range(0,3365):
  res=traindata['winningteam'][i]
  home=traindata['home_team'][i]
  away=traindata['away_team'][i]

  
  traindata['homerank'][i]=table[2][home]
  traindata['awayrank'][i]=table[2][away]

  rank_upgrade(home,away,res)

  if(traindata['section'][i]==34 and traindata['section'][i+1]==1):
    print(table)
    rank_reset()
!pip install numexpr
#直近２、３、５
traindata['home_n3']=0.0
traindata['away_n3']=0.0
for i in range(len(traindata)):
  hex_h = traindata[(traindata['section'][i]-3<=traindata['section'])&(traindata['section']<traindata['section'][i])&(traindata['home_team']==traindata['home_team'][i])&(traindata['year']==traindata['year'][i])]
  hex_a = traindata[(traindata['section'][i]-3<=traindata['section'])&(traindata['section']<traindata['section'][i])
                                     &(traindata['away_team']==traindata['home_team'][i])&(traindata['year']==traindata['year'][i])]
  aex_h = traindata[(traindata['section'][i]-3<=traindata['section'])&(traindata['section']<traindata['section'][i])
                                     &(traindata['home_team']==traindata['away_team'][i])&(traindata['year']==traindata['year'][i])]
  aex_a = traindata[(traindata['section'][i]-3<=traindata['section'])&(traindata['section']<traindata['section'][i])
                                     &(traindata['away_team']==traindata['away_team'][i])&(traindata['year']==traindata['year'][i])]
  a = pd.DataFrame([(hex_h['winningteam']*-1+2).sum(),(hex_a['winningteam']).sum()]).fillna(0)
  b = pd.DataFrame([(aex_h['winningteam']*-1+2).sum(),(aex_a['winningteam']).sum()]).fillna(0)
  traindata['home_n3'][i]=a.sum()/3
  traindata['away_n3'][i]=b.sum()/3
traindata['home_n5']=0.0
traindata['away_n5']=0.0
for i in range(len(traindata)):
  hex_h = traindata[(traindata['section'][i]-5<=traindata['section'])&(traindata['section']<traindata['section'][i])&(traindata['home_team']==traindata['home_team'][i])&(traindata['year']==traindata['year'][i])]
  hex_a = traindata[(traindata['section'][i]-5<=traindata['section'])&(traindata['section']<traindata['section'][i])
                                     &(traindata['away_team']==traindata['home_team'][i])&(traindata['year']==traindata['year'][i])]
  aex_h = traindata[(traindata['section'][i]-5<=traindata['section'])&(traindata['section']<traindata['section'][i])
                                     &(traindata['home_team']==traindata['away_team'][i])&(traindata['year']==traindata['year'][i])]
  aex_a = traindata[(traindata['section'][i]-5<=traindata['section'])&(traindata['section']<traindata['section'][i])
                                     &(traindata['away_team']==traindata['away_team'][i])&(traindata['year']==traindata['year'][i])]
  a = pd.DataFrame([(hex_h['winningteam']*-1+2).sum(),(hex_a['winningteam']).sum()]).fillna(0)
  b = pd.DataFrame([(aex_h['winningteam']*-1+2).sum(),(aex_a['winningteam']).sum()]).fillna(0)
  traindata['home_n5'][i]=a.sum()/5
  traindata['away_n5'][i]=b.sum()/5

traindata['home_n2']=0.0
traindata['away_n2']=0.0
for i in range(len(traindata)):
  hex_h = traindata[(traindata['section'][i]-2<=traindata['section'])&(traindata['section']<traindata['section'][i])&(traindata['home_team']==traindata['home_team'][i])&(traindata['year']==traindata['year'][i])]
  hex_a = traindata[(traindata['section'][i]-2<=traindata['section'])&(traindata['section']<traindata['section'][i])
                                     &(traindata['away_team']==traindata['home_team'][i])&(traindata['year']==traindata['year'][i])]
  aex_h = traindata[(traindata['section'][i]-2<=traindata['section'])&(traindata['section']<traindata['section'][i])
                                     &(traindata['home_team']==traindata['away_team'][i])&(traindata['year']==traindata['year'][i])]
  aex_a = traindata[(traindata['section'][i]-2<=traindata['section'])&(traindata['section']<traindata['section'][i])
                                     &(traindata['away_team']==traindata['away_team'][i])&(traindata['year']==traindata['year'][i])]
  a = pd.DataFrame([(hex_h['winningteam']*-1+2).sum(),(hex_a['winningteam']).sum()]).fillna(0)
  b = pd.DataFrame([(aex_h['winningteam']*-1+2).sum(),(aex_a['winningteam']).sum()]).fillna(0)
  traindata['home_n2'][i]=a.sum()/2
  traindata['away_n2'][i]=b.sum()/2

#不快指数
traindata['不快指数']=0.0
for i in range(len(traindata)):
  T=traindata['temperature'][i]
  H=traindata['humidity'][i]
  traindata['不快指数'][i]=0.81*T+0.01*H*(0.99*T-14.3)+46.3


traindata['capacity']=0.0
for i in range(len(traindata)):
  traindata['capacity'][i]=stdic[traindata['venue'][i]]/100


#カテゴリデータをエンコードしていく
from sklearn.preprocessing import LabelEncoder

x = pd.DataFrame(list(stadiumdict.values())).iloc[:][0].unique()
'''
for s in x:
  traindata["stadium_" + s] = (traindata["venue"] == s)

'''
#venueのエンコード
le = LabelEncoder()
le.fit(x)
encoded=le.transform(traindata['venue'])
traindata['venue'] = encoded

len(x)
'''
#teamsのエンコード
let = LabelEncoder()
let.fit(teams)
traindata['away_team']=let.transform(traindata['away_team'])
traindata['home_team'] = let.transform(traindata['home_team'])
'''
traindata=traindata.drop(['home_team','away_team'],axis=1)
traindata=traindata.drop(['id','weather','match_date','kick_off_time'],axis=1)
traindata=traindata.drop(['broadcasters'],axis=1)
traindata = traindata.astype('float64')



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:224: SettingWithCopyWarning: 
A value is trying 

In [3]:
#testdataの整形

testdata=pd.read_csv('/content/test-5.csv')
testdata['match_date']=pd.to_datetime(testdata['match_date'])
testdata['kick_off_time']=pd.to_datetime(testdata['kick_off_time'])

#スタジアムの編集
for i in range(len(testdata)):
  testdata['venue'][i]=stadiumdict[testdata['venue'][i]]

toriginal=testdata.copy()
testdata['holiday_flag']=0
for i in range(len(testdata)):
  if(testdata['match_date'][i] in holidaydata['holiday_date'].values):
    testdata['holiday_flag'][i]=1
  else:
    testdata['holiday_flag'][i]=0

testdata['zenjitsu']=0
for i in range(len(testdata)):
  if(testdata.iloc[i]['match_date'].weekday()==0 or testdata.iloc[i]['match_date'].weekday()==1 or (testdata['match_date'][i]-timedelta(days=1) in holidaydata['holiday_date'].values)):
      testdata['zenjitsu'][i]=1

testdata['yokujitsu']=0
for i in range(len(testdata)):
  if(testdata.iloc[i]['match_date'].weekday()==4 or testdata.iloc[i]['match_date'].weekday()==5 or (testdata['match_date'][i]+timedelta(days=1) in holidaydata['holiday_date'].values)):
      testdata['yokujitsu'][i]=1
testdata['homescore']=match_reportsdata['home_team_score'][3366:].values
testdata['awayscore']=match_reportsdata['away_team_score'][3366:].values

testdata['month']=0
testdata['year']=0
testdata['day']=0
testdata['hour']=0
testdata['weekday']=0
for i in range(len(testdata)):
  testdata['month'][i]=testdata.loc[i]['match_date'].month
  testdata['year'][i]=testdata.loc[i]['match_date'].year-2000
  testdata['day'][i]=testdata.loc[i]['match_date'].day
  testdata['hour'][i]=testdata.loc[i]['kick_off_time'].hour
  testdata['weekday'][i]=testdata.loc[i]['match_date'].weekday()
  testdata['section'][i]=int(re.split('第|節',testdata['section'][i])[1])
  testdata['round'][i]=int(re.split('第|日',testdata['round'][i])[1])
testdata['winningteam']=0
for i in range(len(testdata)):
  if(testdata['homescore'][i]>testdata['awayscore'][i]):
    testdata['winningteam'][i]=0
  elif(testdata['homescore'][i]<testdata['awayscore'][i]):
    testdata['winningteam'][i]=2
  else:
    testdata['winningteam'][i]=1



for condition in ["晴", "曇", "雨", "屋内", "のち", "時々", "一時"]:
    testdata["weather_{}".format(condition)] = testdata["weather"].str.contains(condition)
    testdata["weather_その他"] = testdata["weather"].str.contains("雪|雷|霧")

for team in teams:
  testdata["team_" + team] = ((testdata["home_team"] == team)|
                                        (testdata["away_team"] == team))

testdata['homerank'],testdata['awayrank']=0,0
table[1]=0
for i in range(0,458):
  res=testdata['winningteam'][i]
  home=testdata['home_team'][i]
  away=testdata['away_team'][i]

  rank_upgrade(home,away,res)
  testdata['homerank'][i]=table[2][home]
  testdata['awayrank'][i]=table[2][away]
  if(testdata['section'][i]==34 and testdata['section'][i+1]==1):
    print(table)
    rank_reset()
#直近３、５
testdata['home_n3']=0.0
testdata['away_n3']=0.0
for i in range(len(testdata)):
  hex_h = testdata[(testdata['section'][i]-3<=testdata['section'])&(testdata['section']<testdata['section'][i])&(testdata['home_team']==testdata['home_team'][i])&(testdata['year']==testdata['year'][i])]
  hex_a = testdata[(testdata['section'][i]-3<=testdata['section'])&(testdata['section']<testdata['section'][i])
                                     &(testdata['away_team']==testdata['home_team'][i])&(testdata['year']==testdata['year'][i])]
  aex_h = testdata[(testdata['section'][i]-3<=testdata['section'])&(testdata['section']<testdata['section'][i])
                                     &(testdata['home_team']==testdata['away_team'][i])&(testdata['year']==testdata['year'][i])]
  aex_a = testdata[(testdata['section'][i]-3<=testdata['section'])&(testdata['section']<testdata['section'][i])
                                     &(testdata['away_team']==testdata['away_team'][i])&(testdata['year']==testdata['year'][i])]
  a = pd.DataFrame([(hex_h['winningteam']*-1+2).sum(),(hex_a['winningteam']).sum()]).fillna(0)
  b = pd.DataFrame([(aex_h['winningteam']*-1+2).sum(),(aex_a['winningteam']).sum()]).fillna(0)
  testdata['home_n3'][i]=a.sum()/3
  testdata['away_n3'][i]=b.sum()/3
testdata['home_n5']=0.0
testdata['away_n5']=0.0
for i in range(len(testdata)):
  hex_h = testdata[(testdata['section'][i]-5<=testdata['section'])&(testdata['section']<testdata['section'][i])&(testdata['home_team']==testdata['home_team'][i])&(testdata['year']==testdata['year'][i])]
  hex_a = testdata[(testdata['section'][i]-5<=testdata['section'])&(testdata['section']<testdata['section'][i])
                                     &(testdata['away_team']==testdata['home_team'][i])&(testdata['year']==testdata['year'][i])]
  aex_h = testdata[(testdata['section'][i]-5<=testdata['section'])&(testdata['section']<testdata['section'][i])
                                     &(testdata['home_team']==testdata['away_team'][i])&(testdata['year']==testdata['year'][i])]
  aex_a = testdata[(testdata['section'][i]-5<=testdata['section'])&(testdata['section']<testdata['section'][i])
                                     &(testdata['away_team']==testdata['away_team'][i])&(testdata['year']==testdata['year'][i])]
  a = pd.DataFrame([(hex_h['winningteam']*-1+2).sum(),(hex_a['winningteam']).sum()]).fillna(0)
  b = pd.DataFrame([(aex_h['winningteam']*-1+2).sum(),(aex_a['winningteam']).sum()]).fillna(0)
  testdata['home_n5'][i]=a.sum()/5
  testdata['away_n5'][i]=b.sum()/5
testdata['home_n2']=0.0
testdata['away_n2']=0.0
for i in range(len(testdata)):
  hex_h = testdata[(testdata['section'][i]-2<=testdata['section'])&(testdata['section']<testdata['section'][i])&(testdata['home_team']==testdata['home_team'][i])&(testdata['year']==testdata['year'][i])]
  hex_a = testdata[(testdata['section'][i]-2<=testdata['section'])&(testdata['section']<testdata['section'][i])
                                     &(testdata['away_team']==testdata['home_team'][i])&(testdata['year']==testdata['year'][i])]
  aex_h = testdata[(testdata['section'][i]-2<=testdata['section'])&(testdata['section']<testdata['section'][i])
                                     &(testdata['home_team']==testdata['away_team'][i])&(testdata['year']==testdata['year'][i])]
  aex_a = testdata[(testdata['section'][i]-2<=testdata['section'])&(testdata['section']<testdata['section'][i])
                                     &(testdata['away_team']==testdata['away_team'][i])&(testdata['year']==testdata['year'][i])]
  a = pd.DataFrame([(hex_h['winningteam']*-1+2).sum(),(hex_a['winningteam']).sum()]).fillna(0)
  b = pd.DataFrame([(aex_h['winningteam']*-1+2).sum(),(aex_a['winningteam']).sum()]).fillna(0)
  testdata['home_n2'][i]=a.sum()/2
  testdata['away_n2'][i]=b.sum()/2

testdata['不快指数']=0.0
for i in range(len(testdata)):
  T=testdata['temperature'][i]
  H=testdata['humidity'][i]
  testdata['不快指数'][i]=0.81*T+0.01*H*(0.99*T-14.3)+46.3


testdata['capacity']=0
for i in range(len(testdata)):
  testdata['capacity'][i]=stdic[testdata['venue'][i]]/100

'''
for s in x:
  testdata["stadium_" + s] = (testdata["venue"] == s)
'''

testencoded=le.transform(testdata['venue'])
testdata['venue'] = testencoded

'''
testdata['away_team']=let.transform(testdata['away_team'])
testdata['home_team'] = let.transform(testdata['home_team'])
'''
testdata=testdata.drop(['home_team','away_team'],axis=1)

testdata=testdata.drop(['id','weather','match_date','kick_off_time'],axis=1)
testdata=testdata.drop(['broadcasters'],axis=1)


testdata = testdata.astype('float64')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:118: SettingWithCopyWarning: 
A value is trying 

In [7]:
traindata=traindata.drop(['homescore','awayscore'],axis=1)
testdata=testdata.drop(['homescore','awayscore'],axis=1)
traindata=traindata.drop(['不快指数'],axis=1)
testdata=testdata.drop(['不快指数'],axis=1)

In [8]:
traindata.to_csv('/content/drive/MyDrive/traindata.csv',index=False)
testdata.to_csv('/content/drive/MyDrive/testdata.csv',index=False)
toriginal.to_csv('/content/drive/MyDrive/toroginal.csv',index=False)
original.to_csv('/content/drive/MyDrive/original.csv',index=False)
target.to_csv('/content/drive/MyDrive/target.csv',index=False)

In [9]:
stcapa=[]
for i in range(len(traindata)):
  stcapa.append(stdic[original['venue'][i]])
pd.DataFrame(stcapa).to_csv('/content/drive/MyDrive/stcapa.csv',index=False)

In [10]:
tstcapa=[]
for i in range(len(testdata)):
  tstcapa.append(stdic[toriginal['venue'][i]])

pd.DataFrame(tstcapa).to_csv('/content/drive/MyDrive/tstcapa.csv',index=False)